In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

<h2>Extract MNIST data</h2>

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
#suppress warnings
tf.logging.set_verbosity(old_v)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#dir(mnist.train)

In [5]:
#you can print out the label for 1st figure
import numpy as np

unique, counts = np.unique( mnist.train.labels[0] , return_counts=True)
print(unique )
mnist.train.images.shape[1]

[0. 1.]


784

In [6]:
num_train = mnist.train.num_examples #55,000
num_validation = mnist.validation.num_examples #5000
num_test = mnist.test.num_examples #10,000


<h2>Set hyperparameters</h2>

In [7]:
#learning rate
lr = .05
#number of traning steps
num_steps =5000 # longer fitting model cycles 
#number of batch_size
batch_size = 128 # 128*5 - more training data 

#network parameters
n_hidden_1 = 100 # 100 - reduce over fitting 
#n_hidden_2 = 200  # do not use another level to reduce over fitting 
#n_hidden_3 = 300
# num_input = mnist.train.images.shape[1] # no. dimensions/pixels for an image
num_input = 784
num_classes = 10

In [8]:
tf.reset_default_graph()

In [9]:
#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.float32,[None,num_classes],name='Y')

#Layers weight & bias
weights = {
    
    'W1': tf.Variable(tf.truncated_normal([num_input, n_hidden_1], stddev=0.1 ),name='W1'),
    #'W2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], stddev=0.1 ),name='W2'),
    #'W3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]),name='W3'),
    'Wout': tf.Variable(tf.truncated_normal([n_hidden_1, num_classes]),name='Wout')
}

biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden_1]),name='b1'),
    #'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden_2]),name='b2'),
   # 'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b3'),
    'bout': tf.Variable(tf.constant(0.1, shape=[num_classes]),name='bout')
}

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
#define a neural net model
def neural_net(x):
    layer_1_out = tf.nn.relu(  tf.add(tf.matmul(x,weights['W1']),biases['b1']) )
    #layer_2_out = tf.nn.relu( tf.add(tf.matmul(layer_1_out,weights['W2']),biases['b2']) )
    #layer_3_out = tf.nn.relu( tf.add(tf.matmul(layer_2_out,weights['W3']),biases['b3']) )
    out = ( tf.add(tf.matmul(layer_1_out,weights['Wout']),biases['bout'])  )

    return out


In [49]:
#help( tf.nn.softmax_cross_entropy_with_logits_v2)

In [11]:
#predicted labels
logits = neural_net(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y),name='loss')

#define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))  

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        #fetch batch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #run optimization
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
        if i % 100 ==0:
            acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y})
            print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
    
    print("Training finished!")
    
    print("Testing ACcuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

step 0, Accuracy= 0.148
step 100, Accuracy= 0.891
step 200, Accuracy= 0.906
step 300, Accuracy= 0.953
step 400, Accuracy= 0.945
step 500, Accuracy= 0.922
step 600, Accuracy= 0.945
step 700, Accuracy= 0.977
step 800, Accuracy= 0.977
step 900, Accuracy= 0.938
step 1000, Accuracy= 0.984
step 1100, Accuracy= 0.977
step 1200, Accuracy= 0.945
step 1300, Accuracy= 0.945
step 1400, Accuracy= 0.984
step 1500, Accuracy= 0.945
step 1600, Accuracy= 0.961
step 1700, Accuracy= 0.977
step 1800, Accuracy= 0.977
step 1900, Accuracy= 0.984
step 2000, Accuracy= 0.992
step 2100, Accuracy= 0.969
step 2200, Accuracy= 0.984
step 2300, Accuracy= 0.984
step 2400, Accuracy= 0.969
step 2500, Accuracy= 0.992
step 2600, Accuracy= 0.977
step 2700, Accuracy= 0.984
step 2800, Accuracy= 0.992
step 2900, Accuracy= 0.969
step 3000, Accuracy= 0.984
step 3100, Accuracy= 0.984
step 3200, Accuracy= 1.000
step 3300, Accuracy= 0.977
step 3400, Accuracy= 1.000
step 3500, Accuracy= 0.977
step 3600, Accuracy= 0.984
step 3700, Ac

<h2> Thoughts and Finding  </h2>

- Nonlinear activation: This is probably the most important change that allows to give a good performance. Here I use ReLu as activation function on the hidden layer. It seems to capture the nonliearn association between the label and the image feature. 

- Overfitting (no. layers, no. hidden nodes): It is another key aspect to get good performance.With complex NN (i.e. multiple layers 2 or 3 or many nodes in a layer) we may experiment overfitting issue that makes the model fitting well but gives poor predictions

- Amount of Training Information (batch size): It seems the amount of data is not a very restriction to this task since when I try small batch size like 32 samples or 1280 both gives >96% accuracy in prediciton. 

- num of iterations: we need a enough number of training iterations to reach a good prediction performance. E.g. lr =0.05 , num of itertation of 500 is not good comparing to 5000. 

- learning rate: too large learning rate will not make the numerical optimization approximate the optimal solution well (e.g. lr = 10), but with appropriate learning rate around 0.1 is good. Too small learning rate will need large number of iterations and takes longer time.
